In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
import re
from nltk.corpus import stopwords

from nltk.util import ngrams

# Visualization     
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# do not print warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("job_offers_original_2.csv")

In [3]:
#drop all columns and keeps only job_title and job_description
df = df[['job_title','company_name', 'job_description']]

In [4]:
#(?<![A-Z\W])  what precedes is a word character EXCEPT for capital letters
#(?=[A-Z])     and what follows is a capital letter
def sepa(text): 
    text = re.sub(r'(?<![A-Z\W])(?=[A-Z])', ' ', text)
    return(text)

df['job_description']=df['job_description'].apply(sepa)

In [5]:
df['job_description'] = df['job_description'].str.replace('\d+', '') # remove digits
df["job_description"] = df["job_description"].str.lower() #lowercase

In [6]:
df['job_description'] = df['job_description'].str.replace('/', ',')

In [7]:
df['job_description'][0]

" do you want to work on the most pressing problem of our generation?we're building the infrastructure for the net zero transition, and we're looking for brilliant engineers, designers, and data scientists who want to help define a low carbon future.decarbonizing the economy requires a granular, real-time view of where emissions come from and how they might be reduced. we build software to automate the carbon footprinting of supply chains. banks, traders, and manufacturers use our product to tame the complexity of international supply networks, identify the most carbon-intensive parts, and find greener alternatives. we were part of the y combinator summer  batch and have secured backing from the uk government's innovation arm, innovate uk, the nat west accelerator and the london business school incubator.to join carbon chain, you'll be a keen technologist who loves to learn from others. our company is made up of  passionate people with expertise ranging from oil refining to deep learni

In [8]:
#some common words that lead to the skills are experience, you'll have, responsible, are looking for, ability to,
#knowledge of, understanding of
import re
import pandas as pd
import spacy
from spacy.util import filter_spans
from spacy.tokens import Span
from spacy.matcher import Matcher

In [9]:
from spacy import displacy
from IPython.display import HTML, display

In [10]:
#Extract the ads into a list
desc=list(df.job_description)

In [11]:
#Initialise Spacy model
import en_core_web_sm
nlp = en_core_web_sm.load()

In [12]:
def highlight_terms(terms, texts):
    for doc in nlp.pipe(texts):
        for sentence in set([tok.sent for tok in doc if tok.lower_ in terms]):
            text = sentence.text.strip() # break docs into sentence
            markup = re.sub(fr'(?i)\b({"|".join(terms)})\b', r'<strong>\1</strong>', text)
            display(HTML(markup))
            print('-----')

In [13]:
highlight_terms(['knowledge'], desc[:10])

-----


-----


-----


-----


-----


-----


-----


-----


-----


In [14]:
matcher = Matcher(nlp.vocab)
pattern = [{'POS': {'IN': ('NOUN', 'PROPN','PUNCT')}, 'OP': '+'}, {'LOWER': 'knowledge'}]
matcher.add('knowledge_noun', [pattern])

pattern = [{'LOWER': 'knowledge'}, {'POS': 'ADP'}, {'POS': {'IN': ('DET', 'NOUN', 'PROPN','PUNCT','CONJ')}, 'OP': '+'}]
matcher.add('knowledge_adp', [pattern])

In [15]:
def show_extraction(examples, *extractors):
    seen = set()
    for doc in nlp.pipe(examples):
        doc.ents = filter_spans([Span(doc, start, end, label) for extractor in extractors for label, start, end in extractor(doc)])
        for tok in doc:
            if tok.lower_ == 'knowledge':
                sentence = tok.sent
                if sentence.text in seen:
                    continue
                seen.update([sentence.text])
                if not sentence.ents:
                    doc.ents = list(doc.ents) + [Span(doc, tok.i, tok.i+1, 'MISSING')]
                displacy.render(sentence, style='ent', options = {'colors': {'MISSING': 'pink',
                                                                            'KNOWLEDGE': 'lightgreen'}})
                

In [16]:
show_extraction(desc[:20], matcher)

In [17]:
def get_extractions(examples, *extractors):
    # Could use context instead of enumerate
    for idx, doc in enumerate(nlp.pipe(examples, batch_size=100, disable=['ner'])):
        for ent in filter_spans([Span(doc, start, end, label) for extractor in extractors for label, start, end in extractor(doc)]):
            sent = ent.root.sent
            yield ent.text, idx, ent.start, ent.end, ent.label_, sent.start, sent.end

In [18]:
list(get_extractions(desc[:10], matcher))

[('knowledge of the posix,unix,linux ecosystem sql-',
  1,
  334,
  345,
  'knowledge_adp',
  151,
  492),
 ('knowledge of the posix,unix,linux ecosystem sql-',
  2,
  235,
  246,
  'knowledge_adp',
  105,
  393),
 ('knowledge of sql', 3, 285, 288, 'knowledge_adp', 272, 370),
 ('knowledge of the technology landscape',
  3,
  399,
  404,
  'knowledge_adp',
  370,
  508),
 ('insight, data protection officer knowledge',
  6,
  258,
  264,
  'knowledge_noun',
  235,
  442),
 (': sql, python, data infrastructure, sql etl,elt knowledge',
  7,
  292,
  305,
  'knowledge_noun',
  259,
  404),
 ('architecture principles posix,unix,linux ecosystem knowledge',
  9,
  452,
  461,
  'knowledge_noun',
  314,
  580),
 ('knowledge of sql (window functions,',
  9,
  498,
  505,
  'knowledge_adp',
  314,
  580)]

In [19]:
#Put it in a dataframe and join with the job metadata
def extract_df(*extractors, n_max=None, **kwargs):
    if n_max is None:
        n_max = len(df)
    ent_df = pd.DataFrame(list(get_extractions(df[:n_max].job_description, *extractors)),
                          columns=['text', 'docidx', 'start', 'end', 'label', 'sent_start', 'sent_end'])
    return ent_df.merge(df, how='left', left_on='docidx', right_index=True)

In [20]:
%time 
ent_df = extract_df(matcher, n_max=1000)
ent_df.head()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


,text,docidx,start,end,label,sent_start,sent_end,job_title,company_name,job_description
0,"knowledge of the posix,unix,linux ecosystem sql-",1,334,345,knowledge_adp,151,492,Data Engineer,Bumble,"we strongly encourage people of colour, lesbi..."
1,"knowledge of the posix,unix,linux ecosystem sql-",2,235,246,knowledge_adp,105,393,Data Engineer,Bumble,"we strongly encourage people of colour, lesbi..."
2,knowledge of sql,3,285,288,knowledge_adp,272,370,Data Engineer,The Data Shed,"description at the data shed, we've been work..."
3,knowledge of the technology landscape,3,399,404,knowledge_adp,370,508,Data Engineer,The Data Shed,"description at the data shed, we've been work..."
4,"insight, data protection officer knowledge",6,258,264,knowledge_noun,235,442,Data Engineer,Square Enix,job summary:square enix has an internal cloud...


In [21]:
#Aggregate the counts of different texts.

#It's more significant if it happens accross multiple Advertisers/Sources.

def aggregate_df(df, col=['text']):
    return (df
            .groupby(col)
            .agg(n_company=('company_name', 'nunique'),
                 n=('job_title', 'count'))
            .reset_index()
            .sort_values(['n_company','n'], ascending=False)
        )

In [21]:
pd.set_option("display.max_colwidth", -1)

In [22]:
aggregate_df(ent_df).head(10)

,text,n_company,n
129,knowledge of sql,5,15
175,"skills, knowledge",4,19
136,knowledge of the,3,14
51,knowledge of a,3,9
72,knowledge of data,3,7
11,analyses knowledge,2,14
94,"knowledge of drug, device",2,10
132,knowledge of statistics,2,9
176,"skills,knowledge",2,9
116,knowledge of ms,2,8


In [23]:
def showent(docidx, start, end, label, sent_start, sent_end, **kwargs):
    # We don't need to parse it, so just make_doc
    doc = nlp.make_doc(desc[docidx])
    doc.ents = [Span(doc, start, end, label)]
    sent = doc[sent_start:sent_end]
    displacy.render(sent, style='ent')
    
def showent_df(df):
    for idx, row in df.iterrows():
        showent(**row)

In [31]:
ent_df['text'] = ent_df['text'].str.replace('knowledge', '')
ent_df['text'] = ent_df['text'].str.replace('of', '')
ent_df['text'] = ent_df['text'].str.replace(',', ';')

In [32]:
aggregate_df(ent_df)

,text,n_company,n
78,sql,5,15
176,skills;,4,19
85,the,3,14
0,a,3,9
21,data,3,7
...,...,...,...
180,sql skills,1,1
181,sql.·,1,1
182,staff.in depth,1,1
186,technologies•,1,1


In [33]:
df_ent_agg = aggregate_df(ent_df)
len(df_ent_agg)

191

In [34]:
skills=df_ent_agg['text'].tolist()

In [35]:
with open('skills_2.txt', 'w') as f:
    for skill in skills:
        print(skill, file=f)

In [36]:
n_max=10000
for a,b,c in zip(skills[:n_max:3],skills[1:n_max:3],skills[2:n_max:3]):
     print('{:<35}{:<35}{:<}'.format(a,b,c))

  sql                              skills;                              the
  a                                  data                             analyses 
  drug; device                       statistics                       skills;
  ms                               requirements                         excel
database                           product                              sas
tensor flow                          the posix;unix;linux ecosystem sql-  data vendor landscape●
  iso                              -data (hadoop) concepts            data visualisation experience 
  all the                            microst fice (                     sql (window functions;
architecture principles posix;unix;linux ecosystem   data modelling; data cleansing;    excel;
  r;python data                    insights bonus points:               core data modelling; manipulation
  data technologies (sme)            excel;                             sql;no
  the linux;posix ecosystem        busi

In [37]:
read_file = pd.read_csv (r'skills_2.txt')
read_file.to_csv (r'skills_2.csv', index=None)